# Images to Video Converter

将图片序列转换为 MP4 视频 - 专为 Bridge V2 Dataset 设计

## 功能
- ✅ 单目录转换
- ✅ 批量转换多个目录
- ✅ 自动按文件名排序（支持自然排序）
- ✅ 支持多种图片格式
- ✅ 自适应不同数据结构
- ✅ 进度条显示

## 支持的数据结构
- 标准格式：`traj_0/images/` → `00000.jpg, 00001.jpg, ...`
- 自定义格式：`traj0/images0/` → `im_0.jpg, im_1.jpg, ...`
- 其他格式：自动识别任何目录结构

In [4]:
# 导入必要的库
import os
import re
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm

print("✅ 库导入成功！")

✅ 库导入成功！


In [5]:
# 工具函数

def natural_sort_key(s):
    """自然排序，正确处理数字顺序"""
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split('([0-9]+)', str(s))]


def get_image_files(directory):
    """获取目录中所有图片文件"""
    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.webp'}
    image_files = []

    for file in Path(directory).iterdir():
        if file.suffix.lower() in image_extensions:
            image_files.append(file)

    # 按文件名自然排序
    image_files.sort(key=natural_sort_key)
    return image_files


def images_to_video(image_dir, output_path=None, fps=25, codec='mp4v'):
    """
    将图片序列转换为视频
    
    Args:
        image_dir: 图片目录路径
        output_path: 输出视频路径（默认：原目录下的 video.mp4）
        fps: 帧率（默认：25）
        codec: 视频编码（默认：mp4v）
    
    Returns:
        bool: 成功返回 True，失败返回 False
    """
    image_dir = Path(image_dir)

    # 检查目录是否存在
    if not image_dir.exists() or not image_dir.is_dir():
        print(f"❌ 错误：目录不存在或不是有效目录: {image_dir}")
        return False

    # 获取所有图片文件
    image_files = get_image_files(image_dir)

    if len(image_files) == 0:
        print(f"❌ 错误：目录中没有找到图片文件: {image_dir}")
        return False

    print(f"📁 图片目录: {image_dir}")
    print(f"🖼️  找到 {len(image_files)} 张图片")
    print(f"   第一张: {image_files[0].name}")
    print(f"   最后一张: {image_files[-1].name}")

    # 读取第一张图片以获取尺寸
    first_image = cv2.imread(str(image_files[0]))
    if first_image is None:
        print(f"❌ 错误：无法读取图片: {image_files[0]}")
        return False

    height, width = first_image.shape[:2]
    print(f"📐 图片尺寸: {width} x {height}")

    # 设置输出路径
    if output_path is None:
        output_path = image_dir / "video.mp4"
    else:
        output_path = Path(output_path)
        if not output_path.is_absolute():
            output_path = image_dir / output_path

    print(f"🎬 输出视频: {output_path}")
    print(f"⚙️  帧率: {fps} FPS")

    # 创建视频写入器
    fourcc = cv2.VideoWriter_fourcc(*codec)
    video_writer = cv2.VideoWriter(
        str(output_path),
        fourcc,
        fps,
        (width, height)
    )

    if not video_writer.isOpened():
        print(f"❌ 错误：无法创建视频文件: {output_path}")
        return False

    # 写入所有图片
    print("🎥 正在生成视频...")
    for img_path in tqdm(image_files, desc="处理进度", unit="帧"):
        frame = cv2.imread(str(img_path))

        if frame is None:
            print(f"⚠️  警告：跳过无法读取的图片: {img_path}")
            continue

        # 确保图片尺寸一致
        if frame.shape[:2] != (height, width):
            frame = cv2.resize(frame, (width, height))

        video_writer.write(frame)

    video_writer.release()

    # 显示结果
    video_size_mb = output_path.stat().st_size / (1024 * 1024)
    video_duration = len(image_files) / fps

    print("\n✅ 视频生成成功！")
    print(f"📊 统计信息:")
    print(f"   - 总帧数: {len(image_files)}")
    print(f"   - 视频时长: {video_duration:.2f} 秒")
    print(f"   - 文件大小: {video_size_mb:.2f} MB")
    print(f"   - 保存路径: {output_path}")

    return True


def batch_convert(parent_dir, pattern="*", fps=25, recursive=False):
    """
    批量转换多个目录中的图片序列
    
    Args:
        parent_dir: 父目录路径
        pattern: 目录匹配模式（默认：*，匹配所有子目录）
        fps: 视频帧率
        recursive: 是否递归搜索子目录
    
    Returns:
        int: 成功转换的目录数量
    """
    parent_dir = Path(parent_dir)

    if not parent_dir.exists() or not parent_dir.is_dir():
        print(f"❌ 错误：目录不存在或不是有效目录: {parent_dir}")
        return 0

    # 查找匹配的目录
    if recursive:
        directories = [d for d in parent_dir.rglob(pattern) if d.is_dir()]
    else:
        directories = [d for d in parent_dir.glob(pattern) if d.is_dir()]

    directories.sort(key=natural_sort_key)

    if len(directories) == 0:
        print(f"❌ 错误：没有找到匹配 '{pattern}' 的目录")
        print(f"   父目录: {parent_dir}")
        print(f"   提示: 请检查匹配模式是否正确")
        return 0

    print(f"🔍 找到 {len(directories)} 个目录")
    print(f"📁 父目录: {parent_dir}")
    print(f"🎯 匹配模式: {pattern}")
    print(f"⚙️  帧率: {fps} FPS")
    print(f"📂 匹配的目录:")
    for i, d in enumerate(directories[:5], 1):  # 只显示前5个
        print(f"   {i}. {d.relative_to(parent_dir)}")
    if len(directories) > 5:
        print(f"   ... 还有 {len(directories) - 5} 个目录")
    print("=" * 60)

    success_count = 0
    failed_dirs = []

    for i, directory in enumerate(directories, 1):
        print(f"\n[{i}/{len(directories)}] 处理目录: {directory.relative_to(parent_dir)}")
        print("-" * 60)

        try:
            success = images_to_video(directory, fps=fps)
            if success:
                success_count += 1
            else:
                failed_dirs.append(directory)
        except Exception as e:
            print(f"❌ 错误：处理失败 - {e}")
            failed_dirs.append(directory)

        print("-" * 60)

    # 显示总结
    print("\n" + "=" * 60)
    print("📊 批量处理完成！")
    print("=" * 60)
    print(f"✅ 成功: {success_count}/{len(directories)}")

    if failed_dirs:
        print(f"❌ 失败: {len(failed_dirs)}")
        print("失败的目录:")
        for d in failed_dirs:
            print(f"   - {d.relative_to(parent_dir)}")
    else:
        print("🎉 所有目录处理成功！")

    return success_count


print("✅ 函数定义完成！")

✅ 函数定义完成！


---

## 方法 1: 单目录转换

将一个目录中的图片转换为视频

In [8]:
# 配置参数
IMAGE_DIR = "/home/zoyo/Downloads/traj_group0/traj0/images0"  # 修改为你的图片目录
OUTPUT_NAME = "video.mp4"           # 输出视频文件名（可选，默认 video.mp4）
FPS = 25                            # 帧率（默认 25）

# 执行转换
images_to_video(
    image_dir=IMAGE_DIR,
    output_path=OUTPUT_NAME,
    fps=FPS
)

📁 图片目录: /home/zoyo/Downloads/traj_group0/traj0/images0
🖼️  找到 49 张图片
   第一张: im_0.jpg
   最后一张: im_48.jpg
📐 图片尺寸: 640 x 480
🎬 输出视频: /home/zoyo/Downloads/traj_group0/traj0/images0/video.mp4
⚙️  帧率: 25 FPS
🎥 正在生成视频...


处理进度: 100%|██████████| 49/49 [00:00<00:00, 532.71帧/s]


✅ 视频生成成功！
📊 统计信息:
   - 总帧数: 49
   - 视频时长: 1.96 秒
   - 文件大小: 0.74 MB
   - 保存路径: /home/zoyo/Downloads/traj_group0/traj0/images0/video.mp4


True

---

## 方法 2: 批量转换多个目录

一次性转换多个目录中的图片（适合 Bridge V2 Dataset）

In [9]:
# 配置参数
PARENT_DIR = "/home/zoyo/Downloads/traj_group0"  # 父目录路径
PATTERN = "*/images*"            # 匹配模式（自适应 images, images0, images1 等）
FPS = 25                         # 帧率（默认 25）
RECURSIVE = False                # 是否递归搜索子目录

# 执行批量转换
success_count = batch_convert(
    parent_dir=PARENT_DIR,
    pattern=PATTERN,
    fps=FPS,
    recursive=RECURSIVE
)

print(f"\n🎉 完成！成功转换 {success_count} 个目录")

🔍 找到 20 个目录
📁 父目录: /home/zoyo/Downloads/traj_group0
🎯 匹配模式: */images*
⚙️  帧率: 25 FPS
📂 匹配的目录:
   1. traj0/images0
   2. traj1/images0
   3. traj2/images0
   4. traj3/images0
   5. traj4/images0
   ... 还有 15 个目录

[1/20] 处理目录: traj0/images0
------------------------------------------------------------
📁 图片目录: /home/zoyo/Downloads/traj_group0/traj0/images0
🖼️  找到 49 张图片
   第一张: im_0.jpg
   最后一张: im_48.jpg
📐 图片尺寸: 640 x 480
🎬 输出视频: /home/zoyo/Downloads/traj_group0/traj0/images0/video.mp4
⚙️  帧率: 25 FPS
🎥 正在生成视频...


处理进度: 100%|██████████| 49/49 [00:00<00:00, 552.06帧/s]



✅ 视频生成成功！
📊 统计信息:
   - 总帧数: 49
   - 视频时长: 1.96 秒
   - 文件大小: 0.74 MB
   - 保存路径: /home/zoyo/Downloads/traj_group0/traj0/images0/video.mp4
------------------------------------------------------------

[2/20] 处理目录: traj1/images0
------------------------------------------------------------
📁 图片目录: /home/zoyo/Downloads/traj_group0/traj1/images0
🖼️  找到 41 张图片
   第一张: im_0.jpg
   最后一张: im_40.jpg
📐 图片尺寸: 640 x 480
🎬 输出视频: /home/zoyo/Downloads/traj_group0/traj1/images0/video.mp4
⚙️  帧率: 25 FPS
🎥 正在生成视频...


处理进度: 100%|██████████| 41/41 [00:00<00:00, 557.83帧/s]



✅ 视频生成成功！
📊 统计信息:
   - 总帧数: 41
   - 视频时长: 1.64 秒
   - 文件大小: 0.57 MB
   - 保存路径: /home/zoyo/Downloads/traj_group0/traj1/images0/video.mp4
------------------------------------------------------------

[3/20] 处理目录: traj2/images0
------------------------------------------------------------
📁 图片目录: /home/zoyo/Downloads/traj_group0/traj2/images0
🖼️  找到 49 张图片
   第一张: im_0.jpg
   最后一张: im_48.jpg
📐 图片尺寸: 640 x 480
🎬 输出视频: /home/zoyo/Downloads/traj_group0/traj2/images0/video.mp4
⚙️  帧率: 25 FPS
🎥 正在生成视频...


...


✅ 视频生成成功！
📊 统计信息:
   - 总帧数: 49
   - 视频时长: 1.96 秒
   - 文件大小: 0.68 MB
   - 保存路径: /home/zoyo/Downloads/traj_group0/traj17/images0/video.mp4
------------------------------------------------------------

[19/20] 处理目录: traj18/images0
------------------------------------------------------------
📁 图片目录: /home/zoyo/Downloads/traj_group0/traj18/images0
🖼️  找到 49 张图片
   第一张: im_0.jpg
   最后一张: im_48.jpg
📐 图片尺寸: 640 x 480
🎬 输出视频: /home/zoyo/Downloads/traj_group0/traj18/images0/video.mp4
⚙️  帧率: 25 FPS
🎥 正在生成视频...


处理进度: 100%|██████████| 49/49 [00:00<00:00, 560.56帧/s]



✅ 视频生成成功！
📊 统计信息:
   - 总帧数: 49
   - 视频时长: 1.96 秒
   - 文件大小: 0.63 MB
   - 保存路径: /home/zoyo/Downloads/traj_group0/traj18/images0/video.mp4
------------------------------------------------------------

[20/20] 处理目录: traj19/images0
------------------------------------------------------------
📁 图片目录: /home/zoyo/Downloads/traj_group0/traj19/images0
🖼️  找到 49 张图片
   第一张: im_0.jpg
   最后一张: im_48.jpg
📐 图片尺寸: 640 x 480
🎬 输出视频: /home/zoyo/Downloads/traj_group0/traj19/images0/video.mp4
⚙️  帧率: 25 FPS
🎥 正在生成视频...


处理进度: 100%|██████████| 49/49 [00:00<00:00, 550.26帧/s]


✅ 视频生成成功！
📊 统计信息:
   - 总帧数: 49
   - 视频时长: 1.96 秒
   - 文件大小: 0.65 MB
   - 保存路径: /home/zoyo/Downloads/traj_group0/traj19/images0/video.mp4
------------------------------------------------------------

📊 批量处理完成！
✅ 成功: 20/20
🎉 所有目录处理成功！

🎉 完成！成功转换 20 个目录


---
# 以下实验性功能未经过详细测试，谨慎使用！！

## Bridge V2 Dataset - 多种数据结构支持

### 数据结构示例

**标准格式:**
```
dataset/
├── traj_0/images/  → 00000.jpg, 00001.jpg, ...
├── traj_1/images/  → 00000.jpg, 00001.jpg, ...
└── ...
```

**自定义格式:**
```
dataset/
├── traj0/images0/  → im_0.jpg, im_1.jpg, ...
├── traj1/images1/  → im_0.jpg, im_1.jpg, ...
└── ...
```

In [ ]:
# Bridge V2 配置
BRIDGE_V2_DIR = "/path/to/bridge_v2_dataset"  # 修改为你的数据集路径

# ========== 选择一个方案，取消注释即可 ==========

# 方案 1: 标准格式 (traj_0/images/, traj_1/images/, ...)
batch_convert(
    parent_dir=BRIDGE_V2_DIR,
    pattern="*/images",  # 精确匹配 'images' 目录
    fps=25
)

# 方案 2: 自定义格式 (traj0/images0/, traj1/images1/, ...)
# batch_convert(
#     parent_dir=BRIDGE_V2_DIR,
#     pattern="*/images*",  # 匹配 images, images0, images1 等
#     fps=25
# )

# 方案 3: 自适应 - 匹配所有包含 'image' 的目录
# batch_convert(
#     parent_dir=BRIDGE_V2_DIR,
#     pattern="*/*image*",  # 匹配任何包含 'image' 的子目录
#     fps=25
# )

# 方案 4: 递归搜索所有 images 相关目录
# batch_convert(
#     parent_dir=BRIDGE_V2_DIR,
#     pattern="**/images*",  # 递归搜索
#     fps=25,
#     recursive=True
# )

---

## 测试不同的匹配模式

用这个单元格测试你的匹配模式是否正确

In [ ]:
# 测试匹配模式（不实际转换，只查看会匹配哪些目录）
TEST_DIR = "/path/to/dataset"  # 测试目录
TEST_PATTERN = "*/images*"     # 测试匹配模式

test_dir = Path(TEST_DIR)
if test_dir.exists():
    matched = list(test_dir.glob(TEST_PATTERN))
    matched.sort(key=natural_sort_key)
    
    print(f"📁 测试目录: {test_dir}")
    print(f"🎯 匹配模式: {TEST_PATTERN}")
    print(f"✅ 找到 {len(matched)} 个匹配的目录:\n")
    
    for i, d in enumerate(matched, 1):
        # 检查目录中的图片数量
        img_files = get_image_files(d)
        print(f"{i:3d}. {str(d.relative_to(test_dir)):40s} ({len(img_files)} 张图片)")
        if len(img_files) > 0:
            print(f"      示例文件: {img_files[0].name}")
    
    if len(matched) == 0:
        print("❌ 没有找到匹配的目录！")
        print("\n💡 建议:")
        print("   - 检查目录路径是否正确")
        print("   - 尝试不同的匹配模式:")
        print("     - '*' : 所有子目录")
        print("     - '*/images' : 精确匹配 images 目录")
        print("     - '*/images*' : 匹配 images, images0, images1 等")
        print("     - '*/*image*' : 匹配任何包含 image 的目录")
else:
    print(f"❌ 目录不存在: {test_dir}")

In [ ]:
# 转换单个特定目录（如果只想转换某几个）
specific_dirs = [
    "traj0/images0",
    "traj1/images1",
    "traj5/images5",
]

DATASET_ROOT = "/path/to/dataset"

for dir_path in specific_dirs:
    full_path = Path(DATASET_ROOT) / dir_path
    if full_path.exists():
        print(f"\n处理: {dir_path}")
        images_to_video(full_path, fps=25)
    else:
        print(f"⚠️  跳过：{dir_path} 不存在")

---

## 快速测试

测试脚本是否正常工作

In [ ]:
# 创建测试数据
import tempfile
import shutil

# 创建临时目录和测试图片
test_dir = Path(tempfile.mkdtemp())
print(f"创建测试目录: {test_dir}")

# 生成 10 张测试图片（模拟 im_0.jpg, im_1.jpg 格式）
for i in range(10):
    # 创建随机彩色图片
    img = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
    # 在图片上添加文字
    cv2.putText(img, f"Frame {i}", (250, 240), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
    # 保存图片（使用 im_0.jpg 格式）
    cv2.imwrite(str(test_dir / f"im_{i}.jpg"), img)

print(f"✅ 创建了 10 张测试图片 (im_0.jpg 到 im_9.jpg)")

# 转换为视频
print("\n开始转换...")
images_to_video(test_dir, fps=25)

# 检查生成的视频
video_path = test_dir / "video.mp4"
if video_path.exists():
    print(f"\n✅ 测试成功！视频已生成: {video_path}")
    print(f"📁 测试目录: {test_dir}")
    print("\n💡 你可以打开视频查看效果")
    print(f"\n🗑️  测试完成后可以删除: {test_dir}")
else:
    print("❌ 测试失败：视频未生成")

# 取消注释以自动删除测试目录
# shutil.rmtree(test_dir)
# print(f"✅ 已删除测试目录")

---

## 常用配置示例

### 不同帧率

```python
# 低帧率（适合快速预览）
images_to_video("./images", fps=10)

# 标准帧率（默认）
images_to_video("./images", fps=25)

# 高帧率（适合慢动作）
images_to_video("./images", fps=60)
```

### 不同匹配模式

```python
# 精确匹配 'images' 目录
batch_convert("./dataset", pattern="*/images")

# 匹配 images, images0, images1 等
batch_convert("./dataset", pattern="*/images*")

# 匹配任何包含 'image' 的目录
batch_convert("./dataset", pattern="*/*image*")

# 只匹配 traj_ 开头的目录
batch_convert("./dataset", pattern="traj_*")

# 只匹配 traj0, traj1 等（无下划线）
batch_convert("./dataset", pattern="traj[0-9]*")

# 递归搜索所有 images 目录
batch_convert("./dataset", pattern="**/images*", recursive=True)
```

### 支持的图片命名格式

自然排序会正确处理以下格式：
- `00000.jpg, 00001.jpg, 00002.jpg, ...`
- `0.jpg, 1.jpg, 2.jpg, ..., 10.jpg, 11.jpg, ...`
- `im_0.jpg, im_1.jpg, im_2.jpg, ...`
- `frame_0.png, frame_1.png, frame_2.png, ...`
- `image001.jpg, image002.jpg, image003.jpg, ...`

---

## 故障排除

### 问题 1: 找不到图片
- 检查图片格式是否支持（jpg, png, bmp, tiff, webp）
- 确认目录路径正确
- 使用测试匹配模式单元格检查

### 问题 2: 匹配模式不对
- 使用「测试不同的匹配模式」单元格
- 查看实际匹配到哪些目录
- 调整 pattern 参数

### 问题 3: 视频无法播放
- 尝试不同的编码器：`images_to_video(dir, codec='avc1')`
- 确保 opencv-python 版本正确

### 问题 4: 图片顺序不对
- 检查文件命名是否符合排序规则
- 查看输出中的「第一张」和「最后一张」信息
- 推荐命名：00000.jpg, 00001.jpg, ... 或 im_0.jpg, im_1.jpg, ...

### 问题 5: 批量转换失败
- 使用测试单元格检查匹配模式
- 确认子目录中有图片文件
- 查看详细错误信息